In [10]:
import pandas as pd
print('up')

up


In [11]:
bbc = pd.read_csv('https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/bbc-text.csv')
bbc.head()


,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [12]:
from sklearn.model_selection import train_test_split

training_sentences, validation_sentences = train_test_split(bbc['text'], test_size=1-0.8, shuffle=False)
training_labels, validation_labels = train_test_split(bbc['category'], test_size=1-0.8, shuffle=False)

In [23]:
categories = bbc.category
print(categories)

0                tech
1            business
2               sport
3               sport
4       entertainment
            ...      
2220         business
2221         politics
2222    entertainment
2223         politics
2224            sport
Name: category, Length: 2225, dtype: object


In [24]:
text = bbc.text
text

0       tv future in the hands of viewers with home th...
1       worldcom boss  left books alone  former worldc...
2       tigers wary of farrell  gamble  leicester say ...
3       yeading face newcastle in fa cup premiership s...
4       ocean s twelve raids box office ocean s twelve...
                              ...                        
2220    cars pull down us retail figures us retail sal...
2221    kilroy unveils immigration policy ex-chatshow ...
2222    rem announce new glasgow concert us band rem h...
2223    how political squabbles snowball it s become c...
2224    souness delight at euro progress boss graeme s...
Name: text, Length: 2225, dtype: object

In [25]:
def remove_stopwords(sentence):
    """
    Removes a list of stopwords
    
    Args:
        sentence (string): sentence to remove the stopwords from
    
    Returns:
        sentence (string): lowercase sentence without the stopwords
    """
    # List of stopwords
    stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
    
    # Sentence converted to lowercase-only
    sentence = sentence.lower()

    words = sentence.split()
    no_words = [w for w in words if w not in stopwords]
    sentence = " ".join(no_words)

    return sentence

In [26]:
sentences = [remove_stopwords(sentence) for sentence in text]
labels = [label for label in categories]
print(labels)

['tech', 'business', 'sport', 'sport', 'entertainment', 'politics', 'politics', 'sport', 'sport', 'entertainment', 'entertainment', 'business', 'business', 'politics', 'sport', 'business', 'politics', 'sport', 'business', 'tech', 'tech', 'tech', 'sport', 'sport', 'tech', 'sport', 'entertainment', 'tech', 'politics', 'entertainment', 'politics', 'tech', 'entertainment', 'entertainment', 'business', 'politics', 'tech', 'entertainment', 'politics', 'business', 'politics', 'sport', 'business', 'sport', 'tech', 'entertainment', 'politics', 'politics', 'politics', 'business', 'sport', 'politics', 'business', 'business', 'sport', 'politics', 'business', 'sport', 'sport', 'business', 'business', 'sport', 'business', 'sport', 'business', 'tech', 'business', 'entertainment', 'tech', 'business', 'politics', 'business', 'politics', 'sport', 'business', 'tech', 'business', 'sport', 'sport', 'business', 'business', 'sport', 'politics', 'business', 'entertainment', 'politics', 'politics', 'business',

In [13]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_portion = .8

In [28]:
train_size = int(training_portion * len(sentences))

train_sentences = sentences[:train_size]
train_labels = labels[:train_size]

validation_sentences = sentences[train_size:]
validation_labels = labels[train_size:]

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
 

In [19]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded_seq = pad_sequences(training_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)

val_sequences = tokenizer.texts_to_sequences(validation_sentences)
val_padded_seq = pad_sequences(val_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)

In [20]:
import numpy as np
label_tokenizer = Tokenizer()

label_tokenizer.fit_on_texts(bbc['category'])

training_label_seq= label_tokenizer.texts_to_sequences(training_labels)
training_label_seq = np.array(training_label_seq) 

val_label_seq= label_tokenizer.texts_to_sequences(validation_labels)
val_label_seq = np.array(val_label_seq) 

In [24]:
import tensorflow as tf

tf.random.set_seed(123)
    
    ### START CODE HERE
    
model = tf.keras.Sequential([ 
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
        tf.keras.layers.Dense(6, activation='softmax')
    ])
    
model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

model.fit(training_padded_seq, training_label_seq, epochs=100, validation_data=(val_padded_seq, val_label_seq))

Epoch 1/100
56/56 [==============================] - 33s 235ms/step - loss: 1.6711 - accuracy: 0.2472 - val_loss: 1.5922 - val_accuracy: 0.2270
Epoch 2/100
56/56 [==============================] - 9s 169ms/step - loss: 1.3512 - accuracy: 0.4191 - val_loss: 1.2005 - val_accuracy: 0.5124
Epoch 3/100
56/56 [==============================] - 9s 169ms/step - loss: 1.0059 - accuracy: 0.6337 - val_loss: 0.9708 - val_accuracy: 0.6629
Epoch 4/100
56/56 [==============================] - 10s 187ms/step - loss: 0.7153 - accuracy: 0.7421 - val_loss: 0.8028 - val_accuracy: 0.7034
Epoch 5/100
56/56 [==============================] - 14s 256ms/step - loss: 0.5721 - accuracy: 0.7899 - val_loss: 0.6732 - val_accuracy: 0.7685
Epoch 6/100
56/56 [==============================] - 12s 222ms/step - loss: 0.4389 - accuracy: 0.8646 - val_loss: 0.6100 - val_accuracy: 0.7888
Epoch 7/100
56/56 [==============================] - 18s 321ms/step - loss: 0.3554 - accuracy: 0.9039 - val_loss: 0.6212 - val_accuracy: 0

KeyboardInterrupt: 

In [34]:
print(2.8972e-07 < 1e-3)

True
